In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv==0.19.0 tqdm==4.62.2 numpy Pillow==7.0.0 matplotlib==3.4.3 opencv-python==4.6.0.66 opencv-python-headless==4.6.0.66 matplotlib
!pip install scikit-learn==0.24.2 torch==1.12.1 torchvision==0.13.1 pytorch-ignite segmentation-models-pytorch==0.2.0 albumentations==1.0.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.1 MB/s 
     |████████████████████████████████| 2.1 MB 31.3 MB/s 
     |████████████████████████████████| 10.3 MB 55.9 MB/s 
  Attempting uninstall: Pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bokeh 2.3.3 requires pillow>=7.1.0, but you have pillow 7.0.0 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 22.3 MB 1.5 MB/s 
     |████████████████████████████████| 264 kB 67.8 MB/s 
     |████████████████████████████████| 87 kB 7.7 MB/s 
     |████████████████████████████████| 98 kB 10.1 MB/s 
     |████████████████████████████████| 58 kB 7.2 MB/s 
     |████████████████████████████████| 376 kB 47.7 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12422 sha256=3c314ccd2e3807dddb51fb71185c28508e704d39ca0bf1533bf3aca2cdb1a31b
  Stored in directory: /root/.cache/pip/wheels/90/6b/0c/f0ad36d00310e65390b0d4c9218ae6250ac579c92540c9097a
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=7f39cf9dac7e7e762d00d8be3f8ce7de26a9f0e9b9c0efb3872a39ef7ca41981
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4

In [ ]:
## Импорт необходимых библиотек
import torch
import torch.nn as nn

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Metric
from ignite.engine import _prepare_batch

from tqdm import tqdm

import json
import base64
import os
import glob
import numpy as np
import cv2

from PIL import Image
import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from typing import Callable, Tuple, Dict, Any, List, Sequence, Iterator, Optional
from collections import defaultdict
from torchvision.models.detection.faster_rcnn import fasterrcnn_resnet50_fpn, FastRCNNPredictor

from ipywidgets import IntProgress
from IPython.display import display
import time

## Сегментация стен + окна

### Вспомогательные функции

In [ ]:
## PLRC DATASET - класс и необходимые функции для датасета

def get_color_map():
    return {
        "wall": 255,
        "window": 255
    }

def tensor_from_rgb_image(image: np.ndarray) -> torch.Tensor:
    image = np.moveaxis(image, -1, 0)
    image = np.ascontiguousarray(image)
    image = torch.from_numpy(image)
    return image


def tensor_from_mask_image(mask: np.ndarray) -> torch.Tensor:
    if len(mask.shape) == 2:
        mask = np.expand_dims(mask, -1)
    return tensor_from_rgb_image(mask)


class PLRCDataset(Dataset):
    def __init__(self, image_folder, transform, start_index, end_index, mask_folder=None):
        self.image_folder = image_folder
        self.mask_folder = mask_folder
        self.transform = transform

        self.images = PLRCDataset.parse_folder(self.image_folder, start_index, end_index)
        self.color_map = get_color_map()

    @staticmethod
    def parse_folder(path, start, end):
        if path is None:
            return []
        images = glob.glob1(path,  '*.png')
        images.sort()

        return images[start:end]

    @staticmethod
    def load_image(path) -> np.array:
        return cv2.imread(path, 0)

    @staticmethod
    def load_mask(path) -> np.array:
        return cv2.imread(path, 0)

    @staticmethod
    def split_grayscale_mask_into_channels_by_color_map(mask, color_map) -> torch.Tensor:
        masks = []

        for i in color_map.values():
            masks.append(mask == i)

        return torch.cat(masks).float()

    def mask_to_grayscale(self, masks) -> np.ndarray:
        masks = masks.cpu().numpy()

        colors_by_index = list(self.color_map.values())
        img = np.zeros(masks.shape[1:], dtype=np.uint8)

        for i in range(len(masks)):
            img[masks[i] == 1] = colors_by_index[i]

        return img

    def __getitem__(self, index):
        image_name = self.images[index]
        image_path = os.path.join(self.image_folder, image_name)

        image = PLRCDataset.load_image(image_path)

        if self.mask_folder is None:
            # sample = self.transform(image=image)
            # image = sample['image']
            return image_name, tensor_from_mask_image(image).float() / 255.0

        mask_path = os.path.join(self.mask_folder, image_name)
        mask = PLRCDataset.load_mask(mask_path)

        sample = self.transform(image=image, mask=mask)
        image, mask = sample['image'], sample['mask']

        image = tensor_from_mask_image(image)
        image = torch.cat([image, image, image])
        mask = tensor_from_mask_image(mask)

        mask = PLRCDataset.split_grayscale_mask_into_channels_by_color_map(mask, self.color_map)

        return image.float() / 255.0, mask

    def __len__(self):
        return len(self.images)

## PLRC UTILS = утилиты для подсчета

def get_training_augmentation():
    return A.Compose([
        # A.RandomCrop(height=256, width=256, p=1),
        A.Normalize(mean=(0.5,), std=(0.5,)),
        A.VerticalFlip(p=0.5),              
        A.RandomRotate90(p=0.5),
        #A.CLAHE(),
        A.RandomBrightnessContrast(p=0.5)    
        #A.RandomGamma(p=0.5)
    ], p=1)

def get_test_augmentation():
    return A.Compose([
        A.Normalize(mean=(0.5,), std=(0.5,)),
    ], p=1)


def get_data_loader(path, batch_size, n_processes, start_index, end_index, shuffle=True):
    image_path = os.path.join(path, 'image')
    mask_path = os.path.join(path, 'mask')

    dataset = PLRCDataset(image_folder=image_path, mask_folder=mask_path, transform=get_training_augmentation(), 
                          start_index=start_index, end_index=end_index)

    return DataLoader(dataset=dataset, batch_size=batch_size, drop_last=True, num_workers=n_processes, shuffle=shuffle)


def get_train_validation_data_loaders(path, batch_size, n_processes, train_split):
    files_count = len(os.listdir(os.path.join(path, 'image')))

    train_dl = get_data_loader(path, batch_size, n_processes, shuffle=True, start_index=0, end_index=int(files_count*train_split))
    test_dl = get_data_loader(path, batch_size, n_processes, shuffle=False, start_index=int(files_count*train_split), end_index=(files_count-1))
    
    return train_dl, test_dl


## DATA LOSS - функции для подсчета метрик качества обучения нейросетей

class BCESoftDiceLoss:
    def __init__(self, dice_weight=0):
        self.nll_loss = nn.BCEWithLogitsLoss()
        self.dice_weight = dice_weight

    @staticmethod
    def soft_dice(predict, target):
        eps = 1e-15
        batch_size = target.size()[0]

        dice_target = (target == 1).float().view(batch_size, -1)
        dice_predict = torch.sigmoid(predict).view(batch_size, -1)

        inter = torch.sum(dice_predict * dice_target) / batch_size
        union = (torch.sum(dice_predict) + torch.sum(dice_target)) / batch_size + eps

        return (2 * inter.float() + eps) / union.float()

    def __call__(self, predict, target):
        loss = (1.0 - self.dice_weight) * self.nll_loss(predict, target)

        if self.dice_weight:
            loss -= self.dice_weight * torch.log(BCESoftDiceLoss.soft_dice(predict, target))

        return loss


class MultiClassBCESoftDiceLoss:
    def __init__(self, dice_weight=0):
        self.bce_soft_dice = BCESoftDiceLoss(dice_weight)

    def __call__(self, predict, target):
        classes = target.shape[1]
        loss = predict.new_zeros(1)

        for i in range(classes):
            loss += self.bce_soft_dice(predict[:, i].unsqueeze(1), target[:, i].unsqueeze(1))

        return loss[0] / float(classes)


class MultiClassSoftDiceMetric(Metric):
    def __init__(self):
        super(MultiClassSoftDiceMetric, self).__init__()
        self.general_loss = 0

    def reset(self):
        self.general_loss = 0

    def update(self, output):
        predict, target = output

        classes = target.shape[1]
        loss = predict.new_zeros(1)

        for i in range(classes):
            loss += BCESoftDiceLoss.soft_dice(predict[:, i].unsqueeze(1), target[:, i].unsqueeze(1))

        self.general_loss = loss[0] / float(classes)

    def compute(self):
        return self.general_loss

## Функции для обучения

def load_trained_model(model, optimizer, model_path, optimizer_path):
    model.load_state_dict(torch.load(model_path))
    optimizer.load_state_dict(torch.load(optimizer_path))
    print('Load model from: ', model_path)
    print('Load optimizer from: ', optimizer_path)


def save_model(model, optimizer, model_path, optimizer_path, postfix='_'):
    torch.save(model.state_dict(), model_path + postfix)
    torch.save(optimizer.state_dict(), optimizer_path + postfix)


def log_image(image, prefix, epoch, step):
    img = Image.fromarray(image)
    image_name = "%s_%s_%s.png" % (epoch, step, prefix)
    img.save(image_name)

    os.remove(image_name)


def run_test_model(model, evaluate_loader, epoch, device, step=10):
    model.eval()
    count_step = 0

    for idx, batch in enumerate(evaluate_loader):
        if count_step > step:
            break

        x, y = _prepare_batch(batch, device)

        predict = model(x)
        predict = torch.sigmoid(predict) > 0.2

        count_step += len(x)

    model.train()


def run_train(dataset_path, batch_size, n_processes, model_path, optimizer_path, load_pre_model=False,
              device='cpu', lr=0.0001, betas=(0.9, 0.99), weight_decay=0.0004, epochs=10,
              log_interval=20, save_interval=2, train_split=1):

    train_loader, evaluate_loader = get_train_validation_data_loaders(path=dataset_path, batch_size=batch_size,
                                                                      n_processes=n_processes, train_split=train_split)
    model = smp.FPN('resnet50', classes=24)

    if device.startswith('cuda'):
        if not torch.cuda.is_available():
            raise ValueError('CUDA is not available')

        model = model.to(device)
        print('CUDA is used')

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=betas, weight_decay=weight_decay)

    if load_pre_model:
        load_trained_model(model, optimizer, model_path, optimizer_path)

    trainer = create_supervised_trainer(model, optimizer, MultiClassBCESoftDiceLoss(0.7), device=device)
    evaluator = create_supervised_evaluator(model,
                                            metrics={'dice': MultiClassSoftDiceMetric(),
                                                     'nll': Loss(MultiClassBCESoftDiceLoss(0.7))},
                                            device=device)

    desc = "ITERATION - loss: {:.2f}"
    pbar = None

    @trainer.on(Events.EPOCH_STARTED)
    def create_pbar(engine):
        model.train()
        nonlocal pbar
        pbar = tqdm(
            initial=0, leave=False, total=len(train_loader),
            desc=desc.format(0)
        )

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        pbar.close()
        evaluator.run(evaluate_loader)
        metrics = evaluator.state.metrics
        avg_dice = metrics['dice']
        avg_nll = metrics['nll']


        print("Training Results - Epoch: {}  Dice: {:.2f} Avg loss: {:.2f}"
              .format(engine.state.epoch, avg_dice, avg_nll))


        if engine.state.epoch % save_interval == 0:
            save_model(model, optimizer, model_path, optimizer_path, '_' + list(get_color_map().keys())[0]) #str(engine.state.epoch))
            run_test_model(model, evaluate_loader, engine.state.epoch, device)

    @trainer.on(Events.ITERATION_COMPLETED)
    def log_training_loss(engine):

        pbar.desc = desc.format(engine.state.output)
        pbar.update()


    model.train()

    trainer.run(train_loader, max_epochs=epochs)

### Обучение

In [ ]:
run_train(dataset_path="/content/drive/MyDrive/train_dataset_train/train_wall", batch_size=16, n_processes=0,
          model_path='/content/drive/MyDrive/model/model',
          optimizer_path='/content/drive/MyDrive/opt/opt', device='cuda', epochs=30,
          load_pre_model=False)

CUDA is used


Training Results - Epoch: 1  Dice: 0.84 Avg loss: 0.15


Training Results - Epoch: 2  Dice: 0.87 Avg loss: 0.12


Training Results - Epoch: 3  Dice: 0.89 Avg loss: 0.11


Training Results - Epoch: 4  Dice: 0.88 Avg loss: 0.11


...

Training Results - Epoch: 23  Dice: 0.93 Avg loss: 0.08


Training Results - Epoch: 24  Dice: 0.93 Avg loss: 0.08


Training Results - Epoch: 25  Dice: 0.92 Avg loss: 0.08


Training Results - Epoch: 26  Dice: 0.92 Avg loss: 0.09


ITERATION - loss: 0.06:  93%|█████████▎| 143/153 [02:45<00:11,  1.16s/it]